In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


# Load data and select only the required columns
required_columns = ['Open', 'Close', 'High', 'Low']
df = pd.read_csv('/content/data.csv')
required_columns = ['Open ', 'Close ', 'High ', 'Low ']  # Note the trailing spaces
# Handle missing values using ffill() and bfill()
df[required_columns] = df[required_columns].ffill().bfill()  # Changed line

# Drop any remaining rows with nulls (if any)
df.dropna(subset=required_columns, inplace=True)
print(df.head())
df.columns
# # Check if the data contains the required columns
# if not set(required_columns).issubset(df.columns):
#     raise ValueError("Error: Data is missing required columns.")


# Drop any remaining rows with nulls (if any)
df.dropna(subset=required_columns, inplace=True)




         Date     Open     High      Low    Close   Shares Traded   \
0  09-NOV-2015  7788.25  7937.75  7771.70  7915.20     218422388.0   
1  10-NOV-2015  7877.60  7885.10  7772.85  7783.35     170267413.0   
2  11-NOV-2015  7838.80  7847.95  7819.10  7825.00      22380435.0   
3  13-NOV-2015  7762.45  7775.10  7730.90  7762.25     165876819.0   
4  16-NOV-2015  7732.95  7838.85  7714.15  7806.60     154134885.0   

   Turnover (₹ Cr)  
0          9376.17  
1          7153.47  
2          1123.44  
3          7731.55  
4          6871.15  


In [ ]:
# Scale data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[required_columns])
# Define a function to create sequences of the past n days for each row as input
def create_sequences(data, window_size=5):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])  # Target is the next day values
    return np.array(X), np.array(y)

# Create sequences with a 5-day window
X, y = create_sequences(scaled_data, window_size=5)

# Split data into train and test sets
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Define LSTM model with dropout layers for regularization
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(4)  # Predicting open, close, high, low
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with increased epochs and early stopping
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

# Evaluate on test set
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

# Make predictions
predictions = model.predict(X_test)

# Inverse scale the predictions to original values
predictions = scaler.inverse_transform(predictions)
y_test_original = scaler.inverse_transform(y_test)

# Display a few predictions vs actual values
print("Predictions:\n", predictions[:5])
print("Actual Values:\n", y_test_original[:5])

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


89/89 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0151 - val_loss: 0.0010
Epoch 2/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9.8567e-04 - val_loss: 2.6064e-04
Epoch 3/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8.2957e-04 - val_loss: 7.5243e-04
Epoch 4/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7.0285e-04 - val_loss: 2.9703e-04
Epoch 5/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.5464e-04 - val_loss: 4.2150e-04
Epoch 6/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 5.5823e-04 - val_loss: 2.6087e-04
Epoch 7/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4.8917e-04 - val_loss: 2.6243e-04
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5408e-04 
Test Loss: 0.0004215695953462273
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predictions:
 [[17886.031 17972.299 17975.992 17891.477]
 [17922.086 18009.121 18013.58  17928.701]
 [17954.7   18043.229 18047.643 17963.35 ]
 [17941.043 18031.912 18034.93  17951.287]
 [17893.79  17991.678 17993.016 17907.432]]
Actual Values

In [ ]:
import numpy as np

# Example custom data (use actual recent values here)
custom_data = [
    [17800, 17900, 18000, 17750],  # day 1 (open, close, high, low)
    [17900, 17850, 18050, 17780],  # day 2

]

# Convert to a numpy array
custom_data = np.array(custom_data)

# Scale custom data using the scaler fitted on the training data
scaled_custom_data = scaler.transform(custom_data)

# Reshape to the expected input shape (1 sequence of 5 days, 4 features)
custom_sequence = np.array([scaled_custom_data])  # shape (1, 5, 4)

# Make prediction for the next day
custom_prediction = model.predict(custom_sequence)

# Inverse scale to get the actual predicted values
predicted_values = scaler.inverse_transform(custom_prediction)

print("Predicted next day's values (open, close, high, low):")
print(predicted_values[0])


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
Predicted next day's values (open, close, high, low):
[10376.704 10120.959 10047.337 10056.054]


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam

# Initialize the model as imodel
imodel = Sequential()

# First Bidirectional LSTM layer
imodel.add(Bidirectional(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))))
imodel.add(Dropout(0.2))  # Dropout to prevent overfitting

# Second Bidirectional LSTM layer
imodel.add(Bidirectional(LSTM(units=128, return_sequences=True)))
imodel.add(Dropout(0.2))  # Dropout layer after second LSTM

# Third LSTM layer
imodel.add(LSTM(units=128))
imodel.add(Dropout(0.2))  # Dropout layer after third LSTM

# Dense layer for further processing
imodel.add(Dense(64, activation='relu'))

# Dropout layer after Dense layer to prevent overfitting
imodel.add(Dropout(0.5))

# Output layer (4 output values: open, close, high, low)
imodel.add(Dense(4))

# Compile the model with Adam optimizer and mean squared error loss function
imodel.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

# Print the model summary to check the architecture
imodel.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
imodel.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)


Epoch 1/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - loss: 0.0318 - val_loss: 0.0171
Epoch 2/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0064 - val_loss: 0.0025
Epoch 3/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0053 - val_loss: 0.0017
Epoch 4/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0052 - val_loss: 7.2799e-04
Epoch 5/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 0.0047 - val_loss: 0.0027
Epoch 6/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0043 - val_loss: 0.0011
Epoch 7/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0040 - val_loss: 0.0025
Epoch 8/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0038 - val_loss: 0.0022
Epoch 9/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - loss: 0.0037 - val_loss: 5.5262e-04
Epoch 10/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0030 - val_loss: 0.0019
Epoch 11/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.0032 - val_loss: 0.0016
Epoch 12/50
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - l

In [ ]:
# Evaluate on test set
test_loss = imodel.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

# Make predictions
predictions = model.predict(X_test)

# Inverse scale the predictions to original values
predictions = scaler.inverse_transform(predictions)
y_test_original = scaler.inverse_transform(y_test)

# Display a few predictions vs actual values
print("Predictions:\n", predictions[:5])
print("Actual Values:\n", y_test_original[:5])

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0015
Test Loss: 0.002473589265719056
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Predictions:
 [[17886.031 17972.299 17975.992 17891.477]
 [17922.086 18009.121 18013.58  17928.701]
 [17954.7   18043.229 18047.643 17963.35 ]
 [17941.043 18031.912 18034.93  17951.287]
 [17893.79  17991.678 17993.016 17907.432]]
Actual Values:
 [[18118.45 18118.55 18162.6  18063.45]
 [18183.95 18118.3  18201.25 18078.65]
 [18093.35 17891.95 18100.6  17846.15]
 [17877.2  17604.35 17884.75 17493.55]
 [17541.95 17648.95 17709.15 17405.55]]


In [ ]:
# Make prediction for the next day
ccustom_prediction = imodel.predict(custom_sequence)

# Inverse scale to get the actual predicted values
ppredicted_values = scaler.inverse_transform(ccustom_prediction)

print("Predicted next day's values (open, close, high, low):")
print(ppredicted_values[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step
Predicted next day's values (open, close, high, low):
[11941.479 11781.876 11941.417 11785.897]
